In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## iCESM Pi Control TAGGED 
## Comparing (via percentage contribution) of a Tagged Region's Precipitation to the Total Precipitation
## vs. seasonal and annual SAM Quartiles (Upper, Lower, and Difference)
### RAW - Meaning that the seasonal cycle is not removed nor are the variables detrended. (Other Notebooks will explore variations of this)



In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.path as mpath
import datetime
import pandas as pd
from scipy.stats.stats import pearsonr
from scipy.stats import ttest_ind
from scipy import signal
from IPython.display import display
import xarray as xr
import dask as da
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
from dask.array.rechunk import rechunk
from mpl_toolkits.basemap import Basemap, addcyclic
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as colors
import statsmodels.api as sm

#for era
from eofs.xarray import Eof
from eofs.examples import example_data_path
np.seterr(divide='ignore', invalid='ignore')

%matplotlib inline

In [3]:
############# to be set #############

                         
##Load ERA indice .pkl's
PICKLE_PATH = '/glade/work/pgoddard/Clay_AA_Research/AA_Tropical_Teleconnections/Save_Indices_as_CSV_19Nov2018'

##Save figure file path
FIG_PATH = '/glade/work/pgoddard/Clay_AA_Research/AA_Tropical_Teleconnections/Figures_for_Drafts_08May2019'

##Load CTRL netcdf as Xarray
CTRL_DATA_PATH = '/glade/work/pgoddard/Clay_AA_Research/AA_Tropical_Teleconnections/Save_CTRL_LGM_ERA_Xarrays_as_netcdf_31Oct2018'
CTRL_DATA = xr.open_dataset(CTRL_DATA_PATH+'/CTRL_DATA_TAGGED.nc')
print(CTRL_DATA)

CTRL_DATA_PATH2 = '/glade/work/pgoddard/Clay_AA_Research/AA_Tropical_Teleconnections/Save_CTRL_LGM_ERA_Xarrays_as_netcdf_31Oct2018'
CTRL_DATA2 = xr.open_dataset(CTRL_DATA_PATH2+'/CTRL_DATA_TAGGED_VAPOR.nc')
print(CTRL_DATA2)

<xarray.Dataset>
Dimensions:       (lat: 96, lon: 144, time: 912)
Coordinates:
  * time          (time) datetime64[ns] 1701-01-31 1701-02-28 ... 1776-12-31
  * lat           (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon           (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    land_frac     (time, lat, lon) float32 ...
    sst           (time, lat, lon) float32 ...
    msl           (time, lat, lon) float32 ...
    u10           (time, lat, lon) float32 ...
    v10           (time, lat, lon) float32 ...
    t2m           (time, lat, lon) float32 ...
    tp            (time, lat, lon) float32 ...
    ci            (time, lat, lon) float32 ...
    d18O          (time, lat, lon) float32 ...
    d18O_Atl_05   (time, lat, lon) float32 ...
    d18O_Atl_15   (time, lat, lon) float32 ...
    d18O_Atl_25   (time, lat, lon) float32 ...
    d18O_Atl_35   (time, lat, lon) float32 ...
    d18O_Atl_45   (time, lat, lon) float32 ...
    d18O_

In [4]:
#getting variables in right units 
slp_tmon_ctrl = CTRL_DATA.data_vars['msl']/100 #hPa
v10_tmon_ctrl = CTRL_DATA.data_vars['v10'] #in m/s at 10m
u10_tmon_ctrl = CTRL_DATA.data_vars['u10'] #in m/s at 10m
t2m_tmon_ctrl = CTRL_DATA.data_vars['t2m']-273.15 #in C
tp_tmon_ctrl  = CTRL_DATA.data_vars['tp']*60*60*24*1000  #in mm/day
ci_tmon_ctrl  = CTRL_DATA.data_vars['ci'] #in [0-1] fraction of sea ice
sst_tmon_ctrl  = CTRL_DATA.data_vars['sst']-273.15 #Equivalent to SST over ocean in C
d18O_tmon_ctrl = CTRL_DATA.data_vars['d18O'] #per mill
land_tmon_ctrl = CTRL_DATA.data_vars['land_frac']

d18O_V_tmon_ctrl = CTRL_DATA2.data_vars['d18O_V'] #per mill
total_prec_water = CTRL_DATA2.data_vars['H2OV_total_prec_water'] #mm

In [5]:
#sst_tmon_ctrl

In [6]:
#Open pandas dfs of all indices and ASL information
pdo_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_pdo_tmon.pkl')
# pdo_index_ctrl.head(n=5)
pdo_seas_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_pdo_seas.pkl')
# pdo_seas_mean_index_ctrl.head(n=5)
pdo_ann_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_pdo_ann_mean.pkl')
# pdo_ann_mean_index_ctrl.head(n=5)

soi_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_soi_tmon.pkl')
# soi_index_ctrl.head(n=5)
soi_seas_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_soi_seas.pkl')
# soi_seas_mean_index_ctrl.head(n=5)
soi_ann_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_soi_ann_mean.pkl')
# soi_ann_mean_index_ctrl.head(n=5)
soi_12roll_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_soi_12roll_mean.pkl')
# soi_12roll_mean_index_ctrl.head(n=10)
soi_6roll_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_soi_6roll_mean.pkl')
# soi_6roll_mean_index_ctrl.head(n=5)
soi_3roll_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_soi_3roll_mean.pkl')
# soi_3roll_mean_index_ctrl.head(n=5)
soi_band_filt_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_soi_band_filt.pkl')
# soi_band_filt_index_ctrl.head(n=5)

nino34_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_nino34_tmon.pkl')
# nino34_index_ctrl.head(n=5)
nino34_seas_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_nino34_seas.pkl')
# nino34_seas_mean_index_ctrl.head(n=5)
nino34_ann_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_nino34_ann_mean.pkl')
# nino34_ann_mean_index_ctrl.head(n=5)
nino34_12roll_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_nino34_12roll_mean.pkl')
# nino34_12roll_mean_index_ctrl.head(n=10)
nino34_6roll_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_nino34_6roll_mean.pkl')
# nino34_6roll_mean_index_ctrl.head(n=5)
nino34_3roll_mean_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_nino34_3roll_mean.pkl')
# nino34_3roll_mean_index_ctrl.head(n=5)
nino34_band_filt_index_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_nino34_band_filt.pkl')
# nino34_band_filt_index_ctrl.head(n=5)

enso_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_epcs_tmon.pkl')
# enso_indices_ctrl.head(n=5)
enso_seas_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_epcs_seas.pkl')
# enso_seas_mean_indices_ctrl.head(n=5)
enso_ann_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_epcs_ann_mean.pkl')
# enso_ann_mean_indices_ctrl.head(n=5)
enso_12roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_epcs_12roll_mean.pkl')
# enso_12roll_mean_indices_ctrl.head(n=10)
enso_6roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_epcs_6roll_mean.pkl')
# enso_6roll_mean_indices_ctrl.head(n=5)
enso_3roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_epcs_3roll_mean.pkl')
# enso_3roll_mean_indices_ctrl.head(n=5)
enso_band_filt_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_epcs_band_filt.pkl')
# enso_band_filt_indices_ctrl.head(n=5)

SAM_PSAs_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_sam_psas_tmon.pkl')
# SAM_PSAs_indices_ctrl.head(n=5)
SAM_PSAs_seas_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_sam_psas_seas.pkl')
# SAM_PSAs_seas_mean_indices_ctrl.head(n=5)
SAM_PSAs_ann_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_sam_psas_ann_mean.pkl')
print(SAM_PSAs_ann_mean_indices_ctrl.head(n=5))
SAM_PSAs_12roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_sam_psas_12roll_mean.pkl')
# SAM_PSAs_12roll_mean_indices_ctrl.head(n=10)
SAM_PSAs_6roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_sam_psas_6roll_mean.pkl')
# SAM_PSAs_6roll_mean_indices_ctrl.head(n=5)
SAM_PSAs_3roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_sam_psas_3roll_mean.pkl')
# SAM_PSAs_3roll_mean_indices_ctrl.head(n=5)
SAM_PSAs_band_filt_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_sam_psas_band_filt.pkl')
# SAM_PSAs_band_filt_indices_ctrl.head(n=5)

df_d18O_psuedoWDC_tmon_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoWDC_tmon.pkl')
# df_d18O_psuedoWDC_tmon_ctrl.head(n=5)
d18O_psuedoWDC_seas_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoWDC_seas.pkl')
# d18O_psuedoWDC_seas_mean_ctrl.head(n=5)
d18O_psuedoWDC_ann_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoWDC_ann_mean.pkl')
# d18O_psuedoWDC_ann_mean_ctrl.head(n=5)
d18O_psuedoWDC_12roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoWDC_12roll_mean.pkl')
# d18O_psuedoWDC_12roll_mean_ctrl.head(n=10)
d18O_psuedoWDC_6roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoWDC_6roll_mean.pkl')
# d18O_psuedoWDC_6roll_mean_ctrl.head(n=5)
d18O_psuedoWDC_3roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoWDC_3roll_mean.pkl')
# d18O_psuedoWDC_3roll_mean_ctrl.head(n=5)
d18O_psuedoWDC_band_filt_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoWDC_band_filt.pkl')
# d18O_psuedoWDC_band_filt_ctrl.head(n=5)

# df_d18O_psuedowestcore_tmon_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedowestcore_tmon.pkl')
# # df_d18O_psuedowestcore_tmon_ctrl.head(n=5)
# d18O_psuedowestcore_seas_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedowestcore_seas.pkl')
# # d18O_psuedowestcore_seas_mean_ctrl.head(n=5)
# d18O_psuedowestcore_ann_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedowestcore_ann_mean.pkl')
# # d18O_psuedowestcore_ann_mean_ctrl.head(n=5)
# d18O_psuedowestcore_12roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedowestcore_12roll_mean.pkl')
# # d18O_psuedowestcore_12roll_mean_ctrl.head(n=10)
# d18O_psuedowestcore_6roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedowestcore_6roll_mean.pkl')
# # d18O_psuedowestcore_6roll_mean_ctrl.head(n=5)
# d18O_psuedowestcore_3roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedowestcore_3roll_mean.pkl')
# # d18O_psuedowestcore_3roll_mean_ctrl.head(n=5)
# d18O_psuedowestcore_band_filt_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedowestcore_band_filt.pkl')
# # d18O_psuedowestcore_band_filt_ctrl.head(n=5)

# df_d18O_psuedoeastcore_tmon_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoeastcore_tmon.pkl')
# # df_d18O_psuedoeastcore_tmon_ctrl.head(n=5)
# d18O_psuedoeastcore_seas_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoeastcore_seas.pkl')
# # d18O_psuedoeastcore_seas_mean_ctrl.head(n=5)
# d18O_psuedoeastcore_ann_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoeastcore_ann_mean.pkl')
# # d18O_psuedoeastcore_ann_mean_ctrl.head(n=5)
# d18O_psuedoeastcore_12roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoeastcore_12roll_mean.pkl')
# # d18O_psuedoeastcore_12roll_mean_ctrl.head(n=10)
# d18O_psuedoeastcore_6roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoeastcore_6roll_mean.pkl')
# # d18O_psuedoeastcore_6roll_mean_ctrl.head(n=5)
# d18O_psuedoeastcore_3roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoeastcore_3roll_mean.pkl')
# # d18O_psuedoeastcore_3roll_mean_ctrl.head(n=5)
# d18O_psuedoeastcore_band_filt_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_psuedoeastcore_band_filt.pkl')
# # d18O_psuedoeastcore_band_filt_ctrl.head(n=5)

# d18O_wais_pcs_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_wais_pcs_tmon.pkl')
# # d18O_wais_pcs_ctrl.head(n=5)
# d18O_wais_pcs_seas_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_wais_pcs_seas.pkl')
# # d18O_wais_pcs_seas_mean_ctrl.head(n=5)
# d18O_wais_pcs_ann_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_wais_pcs_ann_mean.pkl')
# # d18O_wais_pcs_ann_mean_ctrl.head(n=5)
# d18O_wais_pcs_12roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_wais_pcs_12roll_mean.pkl')
# # d18O_wais_pcs_12roll_mean_ctrl.head(n=12)
# d18O_wais_pcs_6roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_wais_pcs_6roll_mean.pkl')
# # d18O_wais_pcs_6roll_mean_ctrl.head(n=5)
# d18O_wais_pcs_3roll_mean_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_wais_pcs_3roll_mean.pkl')
# # d18O_wais_pcs_3roll_mean_ctrl.head(n=5)
# d18O_wais_pcs_band_filt_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_d18O_wais_pcs_band_filt.pkl')
# # d18O_wais_pcs_band_filt_ctrl.head(n=5)

# monthly_Rel_CP_Anoms_Detrend_Norm_loc_info_ctrl_tagged_df = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_tmon.pkl')
# # monthly_Rel_CP_Anoms_Detrend_Norm_loc_info_ctrl_tagged_df.head(n=5)
# Rel_CP_seas_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_seas_mean.pkl')
# # Rel_CP_seas_mean_indices_ctrl.head(n=5)
# Rel_CP_ann_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_ann_mean.pkl')
# # Rel_CP_ann_mean_indices_ctrl.head(n=5)
# Rel_CP_12roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_12roll_mean.pkl')
# # Rel_CP_12roll_mean_indices_ctrl.head(n=10)
# Rel_CP_6roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_6roll_mean.pkl')
# # Rel_CP_6roll_mean_indices_ctrl.head(n=5)
# Rel_CP_3roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_3roll_mean.pkl')
# # Rel_CP_3roll_mean_indices_ctrl.head(n=5)
# Rel_CP_band_filt_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_band_filt.pkl')
# # Rel_CP_band_filt_indices_ctrl.head(n=5)

# monthly_Abs_CP_Anoms_Detrend_Norm_loc_info_ctrl_tagged_df = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Abs_CP_tmon.pkl')
# # monthly_Abs_CP_Anoms_Detrend_Norm_loc_info_ctrl_tagged_df.head(n=5)
# Abs_CP_seas_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Abs_CP_seas_mean.pkl')
# # Abs_CP_seas_mean_indices_ctrl.head(n=5)
# Abs_CP_ann_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Abs_CP_ann_mean.pkl')
# # Abs_CP_ann_mean_indices_ctrl.head(n=5)
# Abs_CP_12roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Abs_CP_12roll_mean.pkl')
# # Abs_CP_12roll_mean_indices_ctrl.head(n=10)
# Abs_CP_6roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Abs_CP_6roll_mean.pkl')
# # Abs_CP_6roll_mean_indices_ctrl.head(n=5)
# Abs_CP_3roll_mean_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Abs_CP_3roll_mean.pkl')
# # Abs_CP_3roll_mean_indices_ctrl.head(n=5)
# Abs_CP_band_filt_indices_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Abs_CP_band_filt.pkl')
# # Abs_CP_band_filt_indices_ctrl.head(n=5)



#Raw Rel and Abs CP Info .... needed for actual lon lat position and values
# monthly_Rel_CP_raw_ctrl_tagged_df = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_Raw_Info_tmon.pkl')
# monthly_Rel_CP_raw_ctrl_tagged_df.head(n=5)
Rel_CP_djf_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_djf_mean.pkl')
# Rel_CP_djf_mean_indices_raw_ctrl.head(n=5)
Rel_CP_mam_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_mam_mean.pkl')
# Rel_CP_mam_mean_indices_raw_ctrl.head(n=5)
Rel_CP_jja_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_jja_mean.pkl')
# Rel_CP_jja_mean_indices_raw_ctrl.head(n=5)
Rel_CP_son_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_son_mean.pkl')
# Rel_CP_son_mean_indices_raw_ctrl.head(n=5)
Rel_CP_ann_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_ann_mean.pkl')
print(Rel_CP_ann_mean_indices_raw_ctrl.head(n=5))
Rel_CP_12roll_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_12roll_mean.pkl')
# Rel_CP_12roll_mean_indices_raw_ctrl.head(n=10)
Rel_CP_6roll_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_6roll_mean.pkl')
# Rel_CP_6roll_mean_indices_raw_ctrl.head(n=5)
Rel_CP_3roll_mean_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_ASL_CP_raw_info_3roll_mean.pkl')
# Rel_CP_3roll_mean_indices_raw_ctrl.head(n=5)
# Rel_CP_band_filt_indices_raw_ctrl = pd.read_pickle(PICKLE_PATH+'/ctrl_tagged_Rel_CP_Raw_Info_band_filt.pkl')
# # Rel_CP_band_filt_indices_raw_ctrl.head(n=5)


                 SAM      PSA1      PSA2
time                                    
1702-01-01 -0.311936 -0.015404  0.094802
1703-01-01 -0.819279  0.553104  0.060246
1704-01-01 -0.714730  0.496267 -0.208190
1705-01-01 -0.431827  0.106548 -0.660348
1706-01-01  0.391807 -0.572466  0.115213
                vals       lats   lons  sectormean  indice
time                                                      
1702-01-01 -5.477830 -71.052632  197.5  986.772813       0
1703-01-01 -5.095673 -71.052632  195.0  989.296173       1
1704-01-01 -4.715917 -71.052632  205.0  989.053075       2
1705-01-01 -7.214285 -71.052632  205.0  987.087820       3
1706-01-01 -6.000570 -71.052632  207.5  982.878988       4


## Creating the TP Tagged Regions 

In [7]:
#Tagged Variables in right units
#Note, global TP is tp_tmon_ctrl
tp_Atl_05_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_05']*60*60*24*1000  #in mm/day  
tp_Atl_15_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_15']*60*60*24*1000  #in mm/day  
tp_Atl_25_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_25']*60*60*24*1000  #in mm/day  
tp_Atl_35_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_35']*60*60*24*1000  #in mm/day  
tp_Atl_45_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_45']*60*60*24*1000  #in mm/day  
tp_Atl_55_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_55']*60*60*24*1000  #in mm/day  
tp_Atl_65_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_65']*60*60*24*1000  #in mm/day  
tp_Atl_75_tmon_ctrl = CTRL_DATA.data_vars['tp_Atl_75']*60*60*24*1000  #in mm/day  

tp_Pac_05_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_05']*60*60*24*1000  #in mm/day  
tp_Pac_15_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_15']*60*60*24*1000  #in mm/day  
tp_Pac_25_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_25']*60*60*24*1000  #in mm/day  
tp_Pac_35_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_35']*60*60*24*1000  #in mm/day  
tp_Pac_45_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_45']*60*60*24*1000  #in mm/day  
tp_Pac_55_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_55']*60*60*24*1000  #in mm/day  
tp_Pac_65_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_65']*60*60*24*1000  #in mm/day  
tp_Pac_75_tmon_ctrl = CTRL_DATA.data_vars['tp_Pac_75']*60*60*24*1000  #in mm/day  

tp_Ind_05_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_05']*60*60*24*1000  #in mm/day  
tp_Ind_15_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_15']*60*60*24*1000  #in mm/day  
tp_Ind_25_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_25']*60*60*24*1000  #in mm/day  
tp_Ind_35_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_35']*60*60*24*1000  #in mm/day  
tp_Ind_45_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_45']*60*60*24*1000  #in mm/day  
tp_Ind_55_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_55']*60*60*24*1000  #in mm/day  
tp_Ind_65_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_65']*60*60*24*1000  #in mm/day  
tp_Ind_75_tmon_ctrl = CTRL_DATA.data_vars['tp_Ind_75']*60*60*24*1000  #in mm/day  

tp_AA_ice_tmon_ctrl = CTRL_DATA.data_vars['tp_AA_ice']*60*60*24*1000  #in mm/day  
tp_AA_land_tmon_ctrl = CTRL_DATA.data_vars['tp_AA_land']*60*60*24*1000  #in mm/day  

## Turn monthly data into Seasonal and Annual mean Anomalies (detrend and remove climatology then seasonal and annual means)
## Not doing!  Note, not changing variable names


In [8]:
#params to be set
time_lo     = '1702-01-01' #Skipped first year
time_hi    = '1776-12-31' 
# shem_xhi = 
# shem_xlo = 
shem_yhi = 0 #Had to reverse hi and lo lat values for ERA
shem_ylo = -90  #Had to reverse hi and lo lat values for ERA


In [9]:
#Detrend monthly data
time_coords = tp_Atl_05_tmon_ctrl.sel(time=slice(time_lo,time_hi)).coords['time']
lat_coords  = tp_Atl_05_tmon_ctrl.sel(lat=slice(shem_ylo,shem_yhi)).coords['lat']
lon_coords  = tp_Atl_05_tmon_ctrl.coords['lon']

#Detrending the total precip data
tp_Atl_05_tmon_detrended_ctrl = xr.DataArray(tp_Atl_05_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Atl_15_tmon_detrended_ctrl = xr.DataArray(tp_Atl_15_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Atl_25_tmon_detrended_ctrl = xr.DataArray(tp_Atl_25_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))
tp_Atl_35_tmon_detrended_ctrl = xr.DataArray(tp_Atl_35_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Atl_45_tmon_detrended_ctrl = xr.DataArray(tp_Atl_45_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Atl_55_tmon_detrended_ctrl = xr.DataArray(tp_Atl_55_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))
tp_Atl_65_tmon_detrended_ctrl = xr.DataArray(tp_Atl_65_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Atl_75_tmon_detrended_ctrl = xr.DataArray(tp_Atl_75_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           


tp_Pac_05_tmon_detrended_ctrl = xr.DataArray(tp_Pac_05_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Pac_15_tmon_detrended_ctrl = xr.DataArray(tp_Pac_15_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Pac_25_tmon_detrended_ctrl = xr.DataArray(tp_Pac_25_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))   
tp_Pac_35_tmon_detrended_ctrl = xr.DataArray(tp_Pac_35_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Pac_45_tmon_detrended_ctrl = xr.DataArray(tp_Pac_45_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Pac_55_tmon_detrended_ctrl = xr.DataArray(tp_Pac_55_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))
tp_Pac_65_tmon_detrended_ctrl = xr.DataArray(tp_Pac_65_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Pac_75_tmon_detrended_ctrl = xr.DataArray(tp_Pac_75_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           


tp_Ind_05_tmon_detrended_ctrl = xr.DataArray(tp_Ind_05_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Ind_15_tmon_detrended_ctrl = xr.DataArray(tp_Ind_15_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Ind_25_tmon_detrended_ctrl = xr.DataArray(tp_Ind_25_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))  
tp_Ind_35_tmon_detrended_ctrl = xr.DataArray(tp_Ind_35_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Ind_45_tmon_detrended_ctrl = xr.DataArray(tp_Ind_45_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Ind_55_tmon_detrended_ctrl = xr.DataArray(tp_Ind_55_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))
tp_Ind_65_tmon_detrended_ctrl = xr.DataArray(tp_Ind_65_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_Ind_75_tmon_detrended_ctrl = xr.DataArray(tp_Ind_75_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           


tp_AA_land_tmon_detrended_ctrl = xr.DataArray(tp_AA_land_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           
tp_AA_ice_tmon_detrended_ctrl = xr.DataArray(tp_AA_ice_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))           


#Global/Total TP variable
tp_Total_tmon_detrended_ctrl = xr.DataArray(tp_tmon_ctrl.sel(time=slice(time_lo,time_hi),lat=slice(shem_ylo,shem_yhi)),
                                                 coords={'time': time_coords, 'lat': lat_coords,'lon': lon_coords}, dims=('time','lat','lon'))    

In [10]:
#NOT REMOVING CLIMATOLOGY EITHER
#Total Precip from each tagged region with climatology remove (detrended from above)
tp_Atl_05_climatology_ctrl = tp_Atl_05_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_05_tmon_raw_ctrl = tp_Atl_05_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_05_climatology_ctrl
# tp_Atl_05_tmon_raw_ctrl

tp_Atl_15_climatology_ctrl = tp_Atl_15_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_15_tmon_raw_ctrl = tp_Atl_15_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_15_climatology_ctrl
# tp_Atl_15_tmon_raw_ctrl

tp_Atl_25_climatology_ctrl = tp_Atl_25_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_25_tmon_raw_ctrl = tp_Atl_25_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_25_climatology_ctrl
# tp_Atl_25_tmon_raw_ctrl

tp_Atl_35_climatology_ctrl = tp_Atl_35_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_35_tmon_raw_ctrl = tp_Atl_35_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_35_climatology_ctrl
# tp_Atl_35_tmon_raw_ctrl

tp_Atl_45_climatology_ctrl = tp_Atl_45_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_45_tmon_raw_ctrl = tp_Atl_45_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_45_climatology_ctrl
# tp_Atl_45_tmon_raw_ctrl

tp_Atl_55_climatology_ctrl = tp_Atl_55_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_55_tmon_raw_ctrl = tp_Atl_55_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_55_climatology_ctrl
# tp_Atl_55_tmon_raw_ctrl

tp_Atl_65_climatology_ctrl = tp_Atl_65_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_65_tmon_raw_ctrl = tp_Atl_65_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_65_climatology_ctrl
# tp_Atl_65_tmon_raw_ctrl

tp_Atl_75_climatology_ctrl = tp_Atl_75_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Atl_75_tmon_raw_ctrl = tp_Atl_75_tmon_detrended_ctrl#.groupby('time.month') - tp_Atl_75_climatology_ctrl
# tp_Atl_75_tmon_raw_ctrl

tp_Pac_05_climatology_ctrl = tp_Pac_05_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_05_tmon_raw_ctrl = tp_Pac_05_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_05_climatology_ctrl
# tp_Pac_05_tmon_raw_ctrl

tp_Pac_15_climatology_ctrl = tp_Pac_15_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_15_tmon_raw_ctrl = tp_Pac_15_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_15_climatology_ctrl
# tp_Pac_15_tmon_raw_ctrl

tp_Pac_25_climatology_ctrl = tp_Pac_25_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_25_tmon_raw_ctrl = tp_Pac_25_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_25_climatology_ctrl
# tp_Pac_25_tmon_raw_ctrl

tp_Pac_35_climatology_ctrl = tp_Pac_35_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_35_tmon_raw_ctrl = tp_Pac_35_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_35_climatology_ctrl
# tp_Pac_35_tmon_raw_ctrl

tp_Pac_45_climatology_ctrl = tp_Pac_45_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_45_tmon_raw_ctrl = tp_Pac_45_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_45_climatology_ctrl
# tp_Pac_45_tmon_raw_ctrl

tp_Pac_55_climatology_ctrl = tp_Pac_55_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_55_tmon_raw_ctrl = tp_Pac_55_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_55_climatology_ctrl
# tp_Pac_55_tmon_raw_ctrl

tp_Pac_65_climatology_ctrl = tp_Pac_65_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_65_tmon_raw_ctrl = tp_Pac_65_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_65_climatology_ctrl
# tp_Pac_65_tmon_raw_ctrl

tp_Pac_75_climatology_ctrl = tp_Pac_75_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Pac_75_tmon_raw_ctrl = tp_Pac_75_tmon_detrended_ctrl#.groupby('time.month') - tp_Pac_75_climatology_ctrl
# tp_Pac_75_tmon_raw_ctrl

tp_Ind_05_climatology_ctrl = tp_Ind_05_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_05_tmon_raw_ctrl = tp_Ind_05_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_05_climatology_ctrl
# tp_Ind_05_tmon_raw_ctrl

tp_Ind_15_climatology_ctrl = tp_Ind_15_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_15_tmon_raw_ctrl = tp_Ind_15_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_15_climatology_ctrl
# tp_Ind_15_tmon_raw_ctrl

tp_Ind_25_climatology_ctrl = tp_Ind_25_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_25_tmon_raw_ctrl = tp_Ind_25_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_25_climatology_ctrl
# tp_Ind_25_tmon_raw_ctrl

tp_Ind_35_climatology_ctrl = tp_Ind_35_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_35_tmon_raw_ctrl = tp_Ind_35_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_35_climatology_ctrl
# tp_Ind_35_tmon_raw_ctrl

tp_Ind_45_climatology_ctrl = tp_Ind_45_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_45_tmon_raw_ctrl = tp_Ind_45_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_45_climatology_ctrl
# tp_Ind_45_tmon_raw_ctrl

tp_Ind_55_climatology_ctrl = tp_Ind_55_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_55_tmon_raw_ctrl = tp_Ind_55_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_55_climatology_ctrl
# tp_Ind_55_tmon_raw_ctrl

tp_Ind_65_climatology_ctrl = tp_Ind_65_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_65_tmon_raw_ctrl = tp_Ind_65_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_65_climatology_ctrl
# tp_Ind_65_tmon_raw_ctrl

tp_Ind_75_climatology_ctrl = tp_Ind_75_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Ind_75_tmon_raw_ctrl = tp_Ind_75_tmon_detrended_ctrl#.groupby('time.month') - tp_Ind_75_climatology_ctrl
# tp_Ind_75_tmon_raw_ctrl

tp_AA_land_climatology_ctrl = tp_AA_land_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_AA_land_tmon_raw_ctrl = tp_AA_land_tmon_detrended_ctrl#.groupby('time.month') - tp_AA_land_climatology_ctrl
# tp_AA_land_tmon_raw_ctrl

tp_AA_ice_climatology_ctrl = tp_AA_ice_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_AA_ice_tmon_raw_ctrl = tp_AA_ice_tmon_detrended_ctrl#.groupby('time.month') - tp_AA_ice_climatology_ctrl
# tp_AA_ice_tmon_raw_ctrl


#Global/Total TP variable
tp_Total_climatology_ctrl = tp_Total_tmon_detrended_ctrl.groupby('time.month').mean('time')
tp_Total_tmon_raw_ctrl = tp_Total_tmon_detrended_ctrl#.groupby('time.month') - tp_Total_climatology_ctrl

In [11]:
# TP field in Seasonal Means and Annual Means:
# NOTE, tp SEASONAL MEANS ARE CENTERED ON CENTER MONTH; in WDC SEASONAL MEANS ARE NUMBERED BY THE FIRST MONTH

tp_Atl_05_tmon_raw_rolls_ctrl = tp_Atl_05_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_05_detrend_ctrl = tp_Atl_05_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_05_detrend_ctrl = tp_Atl_05_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_05_detrend_ctrl = tp_Atl_05_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_05_detrend_ctrl = tp_Atl_05_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_05_detrend_ctrl = tp_Atl_05_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Atl_15_tmon_raw_rolls_ctrl = tp_Atl_15_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_15_detrend_ctrl = tp_Atl_15_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_15_detrend_ctrl = tp_Atl_15_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_15_detrend_ctrl = tp_Atl_15_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_15_detrend_ctrl = tp_Atl_15_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_15_detrend_ctrl = tp_Atl_15_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Atl_25_tmon_raw_rolls_ctrl = tp_Atl_25_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_25_detrend_ctrl = tp_Atl_25_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_25_detrend_ctrl = tp_Atl_25_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_25_detrend_ctrl = tp_Atl_25_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_25_detrend_ctrl = tp_Atl_25_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_25_detrend_ctrl = tp_Atl_25_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Atl_35_tmon_raw_rolls_ctrl = tp_Atl_35_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_35_detrend_ctrl = tp_Atl_35_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_35_detrend_ctrl = tp_Atl_35_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_35_detrend_ctrl = tp_Atl_35_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_35_detrend_ctrl = tp_Atl_35_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_35_detrend_ctrl = tp_Atl_35_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Atl_45_tmon_raw_rolls_ctrl = tp_Atl_45_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_45_detrend_ctrl = tp_Atl_45_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_45_detrend_ctrl = tp_Atl_45_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_45_detrend_ctrl = tp_Atl_45_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_45_detrend_ctrl = tp_Atl_45_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_45_detrend_ctrl = tp_Atl_45_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Atl_55_tmon_raw_rolls_ctrl = tp_Atl_55_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_55_detrend_ctrl = tp_Atl_55_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_55_detrend_ctrl = tp_Atl_55_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_55_detrend_ctrl = tp_Atl_55_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_55_detrend_ctrl = tp_Atl_55_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_55_detrend_ctrl = tp_Atl_55_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Atl_65_tmon_raw_rolls_ctrl = tp_Atl_65_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_65_detrend_ctrl = tp_Atl_65_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_65_detrend_ctrl = tp_Atl_65_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_65_detrend_ctrl = tp_Atl_65_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_65_detrend_ctrl = tp_Atl_65_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_65_detrend_ctrl = tp_Atl_65_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Atl_75_tmon_raw_rolls_ctrl = tp_Atl_75_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Atl_75_detrend_ctrl = tp_Atl_75_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Atl_75_detrend_ctrl = tp_Atl_75_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Atl_75_detrend_ctrl = tp_Atl_75_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Atl_75_detrend_ctrl = tp_Atl_75_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Atl_75_detrend_ctrl = tp_Atl_75_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')



tp_Pac_05_tmon_raw_rolls_ctrl = tp_Pac_05_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_05_detrend_ctrl = tp_Pac_05_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_05_detrend_ctrl = tp_Pac_05_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_05_detrend_ctrl = tp_Pac_05_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_05_detrend_ctrl = tp_Pac_05_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_05_detrend_ctrl = tp_Pac_05_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Pac_15_tmon_raw_rolls_ctrl = tp_Pac_15_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_15_detrend_ctrl = tp_Pac_15_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_15_detrend_ctrl = tp_Pac_15_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_15_detrend_ctrl = tp_Pac_15_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_15_detrend_ctrl = tp_Pac_15_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_15_detrend_ctrl = tp_Pac_15_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Pac_25_tmon_raw_rolls_ctrl = tp_Pac_25_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_25_detrend_ctrl = tp_Pac_25_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_25_detrend_ctrl = tp_Pac_25_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_25_detrend_ctrl = tp_Pac_25_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_25_detrend_ctrl = tp_Pac_25_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_25_detrend_ctrl = tp_Pac_25_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Pac_35_tmon_raw_rolls_ctrl = tp_Pac_35_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_35_detrend_ctrl = tp_Pac_35_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_35_detrend_ctrl = tp_Pac_35_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_35_detrend_ctrl = tp_Pac_35_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_35_detrend_ctrl = tp_Pac_35_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_35_detrend_ctrl = tp_Pac_35_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Pac_45_tmon_raw_rolls_ctrl = tp_Pac_45_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_45_detrend_ctrl = tp_Pac_45_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_45_detrend_ctrl = tp_Pac_45_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_45_detrend_ctrl = tp_Pac_45_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_45_detrend_ctrl = tp_Pac_45_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_45_detrend_ctrl = tp_Pac_45_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Pac_55_tmon_raw_rolls_ctrl = tp_Pac_55_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_55_detrend_ctrl = tp_Pac_55_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_55_detrend_ctrl = tp_Pac_55_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_55_detrend_ctrl = tp_Pac_55_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_55_detrend_ctrl = tp_Pac_55_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_55_detrend_ctrl = tp_Pac_55_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Pac_65_tmon_raw_rolls_ctrl = tp_Pac_65_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_65_detrend_ctrl = tp_Pac_65_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_65_detrend_ctrl = tp_Pac_65_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_65_detrend_ctrl = tp_Pac_65_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_65_detrend_ctrl = tp_Pac_65_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_65_detrend_ctrl = tp_Pac_65_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Pac_75_tmon_raw_rolls_ctrl = tp_Pac_75_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Pac_75_detrend_ctrl = tp_Pac_75_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Pac_75_detrend_ctrl = tp_Pac_75_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Pac_75_detrend_ctrl = tp_Pac_75_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Pac_75_detrend_ctrl = tp_Pac_75_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Pac_75_detrend_ctrl = tp_Pac_75_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')



tp_Ind_05_tmon_raw_rolls_ctrl = tp_Ind_05_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_05_detrend_ctrl = tp_Ind_05_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_05_detrend_ctrl = tp_Ind_05_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_05_detrend_ctrl = tp_Ind_05_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_05_detrend_ctrl = tp_Ind_05_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_05_detrend_ctrl = tp_Ind_05_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Ind_15_tmon_raw_rolls_ctrl = tp_Ind_15_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_15_detrend_ctrl = tp_Ind_15_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_15_detrend_ctrl = tp_Ind_15_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_15_detrend_ctrl = tp_Ind_15_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_15_detrend_ctrl = tp_Ind_15_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_15_detrend_ctrl = tp_Ind_15_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Ind_25_tmon_raw_rolls_ctrl = tp_Ind_25_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_25_detrend_ctrl = tp_Ind_25_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_25_detrend_ctrl = tp_Ind_25_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_25_detrend_ctrl = tp_Ind_25_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_25_detrend_ctrl = tp_Ind_25_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_25_detrend_ctrl = tp_Ind_25_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Ind_35_tmon_raw_rolls_ctrl = tp_Ind_35_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_35_detrend_ctrl = tp_Ind_35_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_35_detrend_ctrl = tp_Ind_35_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_35_detrend_ctrl = tp_Ind_35_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_35_detrend_ctrl = tp_Ind_35_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_35_detrend_ctrl = tp_Ind_35_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Ind_45_tmon_raw_rolls_ctrl = tp_Ind_45_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_45_detrend_ctrl = tp_Ind_45_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_45_detrend_ctrl = tp_Ind_45_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_45_detrend_ctrl = tp_Ind_45_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_45_detrend_ctrl = tp_Ind_45_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_45_detrend_ctrl = tp_Ind_45_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Ind_55_tmon_raw_rolls_ctrl = tp_Ind_55_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_55_detrend_ctrl = tp_Ind_55_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_55_detrend_ctrl = tp_Ind_55_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_55_detrend_ctrl = tp_Ind_55_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_55_detrend_ctrl = tp_Ind_55_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_55_detrend_ctrl = tp_Ind_55_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Ind_65_tmon_raw_rolls_ctrl = tp_Ind_65_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_65_detrend_ctrl = tp_Ind_65_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_65_detrend_ctrl = tp_Ind_65_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_65_detrend_ctrl = tp_Ind_65_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_65_detrend_ctrl = tp_Ind_65_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_65_detrend_ctrl = tp_Ind_65_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

tp_Ind_75_tmon_raw_rolls_ctrl = tp_Ind_75_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Ind_75_detrend_ctrl = tp_Ind_75_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Ind_75_detrend_ctrl = tp_Ind_75_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Ind_75_detrend_ctrl = tp_Ind_75_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Ind_75_detrend_ctrl = tp_Ind_75_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Ind_75_detrend_ctrl = tp_Ind_75_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')




tp_AA_land_tmon_raw_rolls_ctrl = tp_AA_land_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_AA_land_detrend_ctrl = tp_AA_land_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_AA_land_detrend_ctrl = tp_AA_land_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_AA_land_detrend_ctrl = tp_AA_land_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_AA_land_detrend_ctrl = tp_AA_land_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_AA_land_detrend_ctrl = tp_AA_land_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')


tp_AA_ice_tmon_raw_rolls_ctrl = tp_AA_ice_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_AA_ice_detrend_ctrl = tp_AA_ice_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_AA_ice_detrend_ctrl = tp_AA_ice_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_AA_ice_detrend_ctrl = tp_AA_ice_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_AA_ice_detrend_ctrl = tp_AA_ice_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_AA_ice_detrend_ctrl = tp_AA_ice_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')



tp_Total_tmon_raw_rolls_ctrl = tp_Total_tmon_raw_ctrl.rolling(time=3, min_periods=3, center=True).mean()
djf_tp_raw_Total_detrend_ctrl = tp_Total_tmon_raw_rolls_ctrl[12::12,:]
mam_tp_raw_Total_detrend_ctrl = tp_Total_tmon_raw_rolls_ctrl[3::12,:]
jja_tp_raw_Total_detrend_ctrl = tp_Total_tmon_raw_rolls_ctrl[6::12,:]
son_tp_raw_Total_detrend_ctrl = tp_Total_tmon_raw_rolls_ctrl[9::12,:]
ann_mean_tp_raw_Total_detrend_ctrl = tp_Total_tmon_raw_rolls_ctrl.groupby('time.year').mean('time')

### Find the upper and lower quartile of SAM

In [12]:
#Find SAM values for each of the 8 time periods and put in pd.DataFrame
#Then, select the upper and lower quartile values

d = {'SAM': SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '12').dropna(), 'indice': np.arange(0,len(SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '12').dropna()),1)}
SAM_djf_mean_ctrl = pd.DataFrame(data=d)
# SAM_djf_mean_ctrl

d = {'SAM': SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '03').dropna(), 'indice': np.arange(0,len(SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '03').dropna()),1)}
SAM_mam_mean_ctrl = pd.DataFrame(data=d)
# SAM_mam_mean_ctrl

d = {'SAM': SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '06').dropna(), 'indice': np.arange(0,len(SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '06').dropna()),1)}
SAM_jja_mean_ctrl = pd.DataFrame(data=d)
# SAM_jja_mean_ctrl

d = {'SAM': SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '09').dropna(), 'indice': np.arange(0,len(SAM_PSAs_seas_mean_indices_ctrl.SAM.where(SAM_PSAs_seas_mean_indices_ctrl.Seas_Index == '09').dropna()),1)}
SAM_son_mean_ctrl = pd.DataFrame(data=d)
# SAM_son_mean_ctrl

d = {'SAM': SAM_PSAs_ann_mean_indices_ctrl.SAM.dropna(), 'indice': np.arange(0, len(SAM_PSAs_ann_mean_indices_ctrl.SAM.dropna()),1)}
SAM_ann_mean_ctrl = pd.DataFrame(data=d)
# SAM_ann_mean_ctrl


#create list of indices representing the upper/lower quartiles of 'SAM'
#Note, head is lower (sorted from low to high)
SAM_djf_mean_lower_ctrl = SAM_djf_mean_ctrl.sort_values(by=['SAM']).head(n=(round(SAM_djf_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_djf_mean_lower_ctrl
SAM_djf_mean_upper_ctrl = SAM_djf_mean_ctrl.sort_values(by=['SAM']).tail(n=(round(SAM_djf_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_djf_mean_upper_ctrl

SAM_mam_mean_lower_ctrl = SAM_mam_mean_ctrl.sort_values(by=['SAM']).head(n=(round(SAM_mam_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_mam_mean_lower_ctrl
SAM_mam_mean_upper_ctrl = SAM_mam_mean_ctrl.sort_values(by=['SAM']).tail(n=(round(SAM_mam_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_mam_mean_upper_ctrl

SAM_jja_mean_lower_ctrl = SAM_jja_mean_ctrl.sort_values(by=['SAM']).head(n=(round(SAM_jja_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_jja_mean_lower_ctrl
SAM_jja_mean_upper_ctrl = SAM_jja_mean_ctrl.sort_values(by=['SAM']).tail(n=(round(SAM_jja_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_jja_mean_upper_ctrl

SAM_son_mean_lower_ctrl = SAM_son_mean_ctrl.sort_values(by=['SAM']).head(n=(round(SAM_son_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_son_mean_lower_ctrl
SAM_son_mean_upper_ctrl = SAM_son_mean_ctrl.sort_values(by=['SAM']).tail(n=(round(SAM_son_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_son_mean_upper_ctrl

SAM_ann_mean_lower_ctrl = SAM_ann_mean_ctrl.sort_values(by=['SAM']).head(n=(round(SAM_ann_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_ann_mean_lower_ctrl
SAM_ann_mean_upper_ctrl = SAM_ann_mean_ctrl.sort_values(by=['SAM']).tail(n=(round(SAM_ann_mean_ctrl.indice.size/4))).indice.tolist()
# SAM_ann_mean_upper_ctrl

### get spatial variable from each tagged region (and Global/Total) during Upper or Lower Quartile for TP


### Data is Raw - ie, the seasonal mean has not been removed nor was the original data detrended

In [13]:
#Select TP while index is in upper or lower quartile
#Using new ASL info from 19Nov2018
#djf
djf_tp_at_WDC_Atl_05_w_upper_SAM_ctrl = djf_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_05_w_lower_SAM_ctrl = djf_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Atl_15_w_upper_SAM_ctrl = djf_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_15_w_lower_SAM_ctrl = djf_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Atl_25_w_upper_SAM_ctrl = djf_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_25_w_lower_SAM_ctrl = djf_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_05_w_upper_SAM_ctrl = djf_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_05_w_lower_SAM_ctrl = djf_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_15_w_upper_SAM_ctrl = djf_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_15_w_lower_SAM_ctrl = djf_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_25_w_upper_SAM_ctrl = djf_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_25_w_lower_SAM_ctrl = djf_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_05_w_upper_SAM_ctrl = djf_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_05_w_lower_SAM_ctrl = djf_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_15_w_upper_SAM_ctrl = djf_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_15_w_lower_SAM_ctrl = djf_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_25_w_upper_SAM_ctrl = djf_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_25_w_lower_SAM_ctrl = djf_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_AA_land_w_upper_SAM_ctrl = djf_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_AA_land_w_lower_SAM_ctrl = djf_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_AA_ice_w_upper_SAM_ctrl = djf_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_AA_ice_w_lower_SAM_ctrl = djf_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
mam_tp_at_WDC_Atl_05_w_upper_SAM_ctrl = mam_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_05_w_lower_SAM_ctrl = mam_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Atl_15_w_upper_SAM_ctrl = mam_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_15_w_lower_SAM_ctrl = mam_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Atl_25_w_upper_SAM_ctrl = mam_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_25_w_lower_SAM_ctrl = mam_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_05_w_upper_SAM_ctrl = mam_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_05_w_lower_SAM_ctrl = mam_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_15_w_upper_SAM_ctrl = mam_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_15_w_lower_SAM_ctrl = mam_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_25_w_upper_SAM_ctrl = mam_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_25_w_lower_SAM_ctrl = mam_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_05_w_upper_SAM_ctrl = mam_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_05_w_lower_SAM_ctrl = mam_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_15_w_upper_SAM_ctrl = mam_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_15_w_lower_SAM_ctrl = mam_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_25_w_upper_SAM_ctrl = mam_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_25_w_lower_SAM_ctrl = mam_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_AA_land_w_upper_SAM_ctrl = mam_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_AA_land_w_lower_SAM_ctrl = mam_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_AA_ice_w_upper_SAM_ctrl = mam_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_AA_ice_w_lower_SAM_ctrl = mam_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
jja_tp_at_WDC_Atl_05_w_upper_SAM_ctrl = jja_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_05_w_lower_SAM_ctrl = jja_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Atl_15_w_upper_SAM_ctrl = jja_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_15_w_lower_SAM_ctrl = jja_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Atl_25_w_upper_SAM_ctrl = jja_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_25_w_lower_SAM_ctrl = jja_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_05_w_upper_SAM_ctrl = jja_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_05_w_lower_SAM_ctrl = jja_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_15_w_upper_SAM_ctrl = jja_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_15_w_lower_SAM_ctrl = jja_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_25_w_upper_SAM_ctrl = jja_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_25_w_lower_SAM_ctrl = jja_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_05_w_upper_SAM_ctrl = jja_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_05_w_lower_SAM_ctrl = jja_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_15_w_upper_SAM_ctrl = jja_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_15_w_lower_SAM_ctrl = jja_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_25_w_upper_SAM_ctrl = jja_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_25_w_lower_SAM_ctrl = jja_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_AA_land_w_upper_SAM_ctrl = jja_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_AA_land_w_lower_SAM_ctrl = jja_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_AA_ice_w_upper_SAM_ctrl = jja_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_AA_ice_w_lower_SAM_ctrl = jja_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
son_tp_at_WDC_Atl_05_w_upper_SAM_ctrl = son_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_05_w_lower_SAM_ctrl = son_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Atl_15_w_upper_SAM_ctrl = son_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_15_w_lower_SAM_ctrl = son_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Atl_25_w_upper_SAM_ctrl = son_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_25_w_lower_SAM_ctrl = son_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_05_w_upper_SAM_ctrl = son_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_05_w_lower_SAM_ctrl = son_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_15_w_upper_SAM_ctrl = son_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_15_w_lower_SAM_ctrl = son_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_25_w_upper_SAM_ctrl = son_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_25_w_lower_SAM_ctrl = son_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_05_w_upper_SAM_ctrl = son_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_05_w_lower_SAM_ctrl = son_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_15_w_upper_SAM_ctrl = son_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_15_w_lower_SAM_ctrl = son_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_25_w_upper_SAM_ctrl = son_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_25_w_lower_SAM_ctrl = son_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_AA_land_w_upper_SAM_ctrl = son_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_AA_land_w_lower_SAM_ctrl = son_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_AA_ice_w_upper_SAM_ctrl = son_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_AA_ice_w_lower_SAM_ctrl = son_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
ann_mean_tp_at_WDC_Atl_05_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_05_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Atl_15_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_15_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Atl_25_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_25_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_05_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_05_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_15_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_15_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_25_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_25_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_05_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_05_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_05_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_15_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_15_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_15_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_25_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_25_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_25_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_AA_land_w_upper_SAM_ctrl = ann_mean_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_AA_land_w_lower_SAM_ctrl = ann_mean_tp_raw_AA_land_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_AA_ice_w_upper_SAM_ctrl = ann_mean_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_AA_ice_w_lower_SAM_ctrl = ann_mean_tp_raw_AA_ice_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)


#other lats
djf_tp_at_WDC_Atl_35_w_upper_SAM_ctrl = djf_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_35_w_lower_SAM_ctrl = djf_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Atl_45_w_upper_SAM_ctrl = djf_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_45_w_lower_SAM_ctrl = djf_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Atl_55_w_upper_SAM_ctrl = djf_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_55_w_lower_SAM_ctrl = djf_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Atl_65_w_upper_SAM_ctrl = djf_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_65_w_lower_SAM_ctrl = djf_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Atl_75_w_upper_SAM_ctrl = djf_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Atl_75_w_lower_SAM_ctrl = djf_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_35_w_upper_SAM_ctrl = djf_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_35_w_lower_SAM_ctrl = djf_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_45_w_upper_SAM_ctrl = djf_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_45_w_lower_SAM_ctrl = djf_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_55_w_upper_SAM_ctrl = djf_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_55_w_lower_SAM_ctrl = djf_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_65_w_upper_SAM_ctrl = djf_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_65_w_lower_SAM_ctrl = djf_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Pac_75_w_upper_SAM_ctrl = djf_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Pac_75_w_lower_SAM_ctrl = djf_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_35_w_upper_SAM_ctrl = djf_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_35_w_lower_SAM_ctrl = djf_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_45_w_upper_SAM_ctrl = djf_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_45_w_lower_SAM_ctrl = djf_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_55_w_upper_SAM_ctrl = djf_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_55_w_lower_SAM_ctrl = djf_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_65_w_upper_SAM_ctrl = djf_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_65_w_lower_SAM_ctrl = djf_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)
djf_tp_at_WDC_Ind_75_w_upper_SAM_ctrl = djf_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_at_WDC_Ind_75_w_lower_SAM_ctrl = djf_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_djf_mean_lower_ctrl)

mam_tp_at_WDC_Atl_35_w_upper_SAM_ctrl = mam_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_35_w_lower_SAM_ctrl = mam_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Atl_45_w_upper_SAM_ctrl = mam_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_45_w_lower_SAM_ctrl = mam_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Atl_55_w_upper_SAM_ctrl = mam_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_55_w_lower_SAM_ctrl = mam_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Atl_65_w_upper_SAM_ctrl = mam_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_65_w_lower_SAM_ctrl = mam_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Atl_75_w_upper_SAM_ctrl = mam_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Atl_75_w_lower_SAM_ctrl = mam_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_35_w_upper_SAM_ctrl = mam_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_35_w_lower_SAM_ctrl = mam_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_45_w_upper_SAM_ctrl = mam_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_45_w_lower_SAM_ctrl = mam_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_55_w_upper_SAM_ctrl = mam_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_55_w_lower_SAM_ctrl = mam_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_65_w_upper_SAM_ctrl = mam_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_65_w_lower_SAM_ctrl = mam_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Pac_75_w_upper_SAM_ctrl = mam_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Pac_75_w_lower_SAM_ctrl = mam_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_35_w_upper_SAM_ctrl = mam_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_35_w_lower_SAM_ctrl = mam_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_45_w_upper_SAM_ctrl = mam_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_45_w_lower_SAM_ctrl = mam_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_55_w_upper_SAM_ctrl = mam_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_55_w_lower_SAM_ctrl = mam_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_65_w_upper_SAM_ctrl = mam_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_65_w_lower_SAM_ctrl = mam_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)
mam_tp_at_WDC_Ind_75_w_upper_SAM_ctrl = mam_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_at_WDC_Ind_75_w_lower_SAM_ctrl = mam_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_mam_mean_lower_ctrl)

jja_tp_at_WDC_Atl_35_w_upper_SAM_ctrl = jja_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_35_w_lower_SAM_ctrl = jja_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Atl_45_w_upper_SAM_ctrl = jja_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_45_w_lower_SAM_ctrl = jja_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Atl_55_w_upper_SAM_ctrl = jja_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_55_w_lower_SAM_ctrl = jja_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Atl_65_w_upper_SAM_ctrl = jja_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_65_w_lower_SAM_ctrl = jja_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Atl_75_w_upper_SAM_ctrl = jja_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Atl_75_w_lower_SAM_ctrl = jja_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_35_w_upper_SAM_ctrl = jja_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_35_w_lower_SAM_ctrl = jja_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_45_w_upper_SAM_ctrl = jja_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_45_w_lower_SAM_ctrl = jja_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_55_w_upper_SAM_ctrl = jja_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_55_w_lower_SAM_ctrl = jja_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_65_w_upper_SAM_ctrl = jja_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_65_w_lower_SAM_ctrl = jja_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Pac_75_w_upper_SAM_ctrl = jja_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Pac_75_w_lower_SAM_ctrl = jja_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_35_w_upper_SAM_ctrl = jja_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_35_w_lower_SAM_ctrl = jja_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_45_w_upper_SAM_ctrl = jja_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_45_w_lower_SAM_ctrl = jja_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_55_w_upper_SAM_ctrl = jja_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_55_w_lower_SAM_ctrl = jja_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_65_w_upper_SAM_ctrl = jja_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_65_w_lower_SAM_ctrl = jja_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)
jja_tp_at_WDC_Ind_75_w_upper_SAM_ctrl = jja_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_at_WDC_Ind_75_w_lower_SAM_ctrl = jja_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_jja_mean_lower_ctrl)

son_tp_at_WDC_Atl_35_w_upper_SAM_ctrl = son_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_35_w_lower_SAM_ctrl = son_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Atl_45_w_upper_SAM_ctrl = son_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_45_w_lower_SAM_ctrl = son_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Atl_55_w_upper_SAM_ctrl = son_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_55_w_lower_SAM_ctrl = son_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Atl_65_w_upper_SAM_ctrl = son_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_65_w_lower_SAM_ctrl = son_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Atl_75_w_upper_SAM_ctrl = son_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Atl_75_w_lower_SAM_ctrl = son_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_35_w_upper_SAM_ctrl = son_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_35_w_lower_SAM_ctrl = son_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_45_w_upper_SAM_ctrl = son_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_45_w_lower_SAM_ctrl = son_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_55_w_upper_SAM_ctrl = son_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_55_w_lower_SAM_ctrl = son_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_65_w_upper_SAM_ctrl = son_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_65_w_lower_SAM_ctrl = son_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Pac_75_w_upper_SAM_ctrl = son_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Pac_75_w_lower_SAM_ctrl = son_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_35_w_upper_SAM_ctrl = son_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_35_w_lower_SAM_ctrl = son_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_45_w_upper_SAM_ctrl = son_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_45_w_lower_SAM_ctrl = son_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_55_w_upper_SAM_ctrl = son_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_55_w_lower_SAM_ctrl = son_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_65_w_upper_SAM_ctrl = son_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_65_w_lower_SAM_ctrl = son_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)
son_tp_at_WDC_Ind_75_w_upper_SAM_ctrl = son_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_upper_ctrl)
son_tp_at_WDC_Ind_75_w_lower_SAM_ctrl = son_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(time = SAM_son_mean_lower_ctrl)

ann_mean_tp_at_WDC_Atl_35_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_35_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Atl_45_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_45_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Atl_55_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_55_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Atl_65_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_65_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Atl_75_w_upper_SAM_ctrl = ann_mean_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Atl_75_w_lower_SAM_ctrl = ann_mean_tp_raw_Atl_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_35_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_35_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_45_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_45_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_55_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_55_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_65_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_65_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Pac_75_w_upper_SAM_ctrl = ann_mean_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Pac_75_w_lower_SAM_ctrl = ann_mean_tp_raw_Pac_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_35_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_35_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_35_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_45_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_45_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_45_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_55_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_55_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_55_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_65_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_65_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_65_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)
ann_mean_tp_at_WDC_Ind_75_w_upper_SAM_ctrl = ann_mean_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_at_WDC_Ind_75_w_lower_SAM_ctrl = ann_mean_tp_raw_Ind_75_detrend_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').isel(year = SAM_ann_mean_lower_ctrl)

In [14]:
#Select TP raw for each tagged region while index is in upper or lower quartile

#Global/Total 
djf_tp_raw_Total_w_upper_SAM_ctrl = djf_tp_raw_Total_detrend_ctrl.isel(time = SAM_djf_mean_upper_ctrl)
djf_tp_raw_Total_w_lower_SAM_ctrl = djf_tp_raw_Total_detrend_ctrl.isel(time = SAM_djf_mean_lower_ctrl)
mam_tp_raw_Total_w_upper_SAM_ctrl = mam_tp_raw_Total_detrend_ctrl.isel(time = SAM_mam_mean_upper_ctrl)
mam_tp_raw_Total_w_lower_SAM_ctrl = mam_tp_raw_Total_detrend_ctrl.isel(time = SAM_mam_mean_lower_ctrl)
jja_tp_raw_Total_w_upper_SAM_ctrl = jja_tp_raw_Total_detrend_ctrl.isel(time = SAM_jja_mean_upper_ctrl)
jja_tp_raw_Total_w_lower_SAM_ctrl = jja_tp_raw_Total_detrend_ctrl.isel(time = SAM_jja_mean_lower_ctrl)
son_tp_raw_Total_w_upper_SAM_ctrl = son_tp_raw_Total_detrend_ctrl.isel(time = SAM_son_mean_upper_ctrl)
son_tp_raw_Total_w_lower_SAM_ctrl = son_tp_raw_Total_detrend_ctrl.isel(time = SAM_son_mean_lower_ctrl)
ann_mean_tp_raw_Total_w_upper_SAM_ctrl = ann_mean_tp_raw_Total_detrend_ctrl.isel(year = SAM_ann_mean_upper_ctrl)
ann_mean_tp_raw_Total_w_lower_SAM_ctrl = ann_mean_tp_raw_Total_detrend_ctrl.isel(year = SAM_ann_mean_lower_ctrl)

## Print out the upper, lower, and difference of TP at the WDC for the 10 degree lats in each basin for each season and annual

In [15]:
print('Mean Total Precip at WDC during SAM Quartiles')
print('Atl 05 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 15 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 25 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 35 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 45 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 55 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 65 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 75 DJF SAM Upper =',np.round(djf_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 DJF SAM Lower =',np.round(djf_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 DJF SAM Diff =',np.round(djf_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 05 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 15 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 25 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 35 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 45 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 55 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 65 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 75 DJF SAM Upper =',np.round(djf_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 DJF SAM Lower =',np.round(djf_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 DJF SAM Diff =',np.round(djf_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 05 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 15 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 25 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 35 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 45 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 55 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 65 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 75 DJF SAM Upper =',np.round(djf_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 DJF SAM Lower =',np.round(djf_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 DJF SAM Diff =',np.round(djf_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Land DJF SAM Upper =',np.round(djf_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land DJF SAM Lower =',np.round(djf_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land DJF SAM Diff =',np.round(djf_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Ice DJF SAM Upper =',np.round(djf_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice DJF SAM Lower =',np.round(djf_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice DJF SAM Diff =',np.round(djf_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values - djf_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('')
print('Atl 05 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 15 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 25 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 35 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 45 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 55 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 65 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 75 MAM SAM Upper =',np.round(mam_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 MAM SAM Lower =',np.round(mam_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 MAM SAM Diff =',np.round(mam_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 05 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 15 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 25 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 35 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 45 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 55 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 65 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 75 MAM SAM Upper =',np.round(mam_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 MAM SAM Lower =',np.round(mam_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 MAM SAM Diff =',np.round(mam_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 05 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 15 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 25 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 35 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 45 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 55 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 65 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 75 MAM SAM Upper =',np.round(mam_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 MAM SAM Lower =',np.round(mam_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 MAM SAM Diff =',np.round(mam_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Land MAM SAM Upper =',np.round(mam_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land MAM SAM Lower =',np.round(mam_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land MAM SAM Diff =',np.round(mam_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Ice MAM SAM Upper =',np.round(mam_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice MAM SAM Lower =',np.round(mam_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice MAM SAM Diff =',np.round(mam_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values - mam_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('')
print('Atl 05 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 15 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 25 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 35 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 45 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 55 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 65 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 75 JJA SAM Upper =',np.round(jja_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 JJA SAM Lower =',np.round(jja_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 JJA SAM Diff =',np.round(jja_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 05 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 15 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 25 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 35 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 45 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 55 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 65 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 75 JJA SAM Upper =',np.round(jja_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 JJA SAM Lower =',np.round(jja_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 JJA SAM Diff =',np.round(jja_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 05 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 15 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 25 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 35 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 45 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 55 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 65 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 75 JJA SAM Upper =',np.round(jja_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 JJA SAM Lower =',np.round(jja_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 JJA SAM Diff =',np.round(jja_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Land JJA SAM Upper =',np.round(jja_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land JJA SAM Lower =',np.round(jja_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land JJA SAM Diff =',np.round(jja_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Ice JJA SAM Upper =',np.round(jja_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice JJA SAM Lower =',np.round(jja_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice JJA SAM Diff =',np.round(jja_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values - jja_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('')
print('Atl 05 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 15 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 25 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 35 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 45 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 55 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 65 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 75 SON SAM Upper =',np.round(son_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 SON SAM Lower =',np.round(son_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 SON SAM Diff =',np.round(son_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 05 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 15 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 25 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 35 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 45 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 55 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 65 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 75 SON SAM Upper =',np.round(son_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 SON SAM Lower =',np.round(son_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 SON SAM Diff =',np.round(son_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 05 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 15 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 25 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 35 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 45 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 55 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 65 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 75 SON SAM Upper =',np.round(son_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 SON SAM Lower =',np.round(son_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 SON SAM Diff =',np.round(son_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Land SON SAM Upper =',np.round(son_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land SON SAM Lower =',np.round(son_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land SON SAM Diff =',np.round(son_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Ice SON SAM Upper =',np.round(son_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice SON SAM Lower =',np.round(son_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice SON SAM Diff =',np.round(son_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values - son_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2))

print('')
print('Atl 05 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 05 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 15 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 15 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 25 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 25 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 35 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 35 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 45 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 45 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 55 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 55 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 65 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 65 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Atl 75 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Atl 75 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 05 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 05 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 15 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 15 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 25 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 25 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 35 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 35 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 45 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 45 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 55 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 55 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 65 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 65 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Pac 75 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Pac 75 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 05 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 05 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 15 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 15 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 25 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 25 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 35 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 35 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 45 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 45 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 55 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 55 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 65 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 65 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('Ind 75 ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', Ind 75 ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Land ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Land ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0).values,2))
print('AA Ice ANN MEAN SAM Upper =',np.round(ann_mean_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice ANN MEAN SAM Lower =',np.round(ann_mean_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2),
      ', AA Ice ANN MEAN SAM Diff =',np.round(ann_mean_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0).values - ann_mean_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0).values,2))

Mean Total Precip at WDC during SAM Quartiles
Atl 05 DJF SAM Upper = 0.0 , Atl 05 DJF SAM Lower = 0.0 , Atl 05 DJF SAM Diff = -0.0
Atl 15 DJF SAM Upper = 0.0 , Atl 15 DJF SAM Lower = 0.0 , Atl 15 DJF SAM Diff = -0.0
Atl 25 DJF SAM Upper = 0.0 , Atl 25 DJF SAM Lower = 0.0 , Atl 25 DJF SAM Diff = -0.0
Atl 35 DJF SAM Upper = 0.01 , Atl 35 DJF SAM Lower = 0.01 , Atl 35 DJF SAM Diff = -0.0
Atl 45 DJF SAM Upper = 0.01 , Atl 45 DJF SAM Lower = 0.01 , Atl 45 DJF SAM Diff = 0.0
Atl 55 DJF SAM Upper = 0.0 , Atl 55 DJF SAM Lower = 0.01 , Atl 55 DJF SAM Diff = -0.0
Atl 65 DJF SAM Upper = 0.0 , Atl 65 DJF SAM Lower = 0.0 , Atl 65 DJF SAM Diff = -0.0
Atl 75 DJF SAM Upper = 0.0 , Atl 75 DJF SAM Lower = 0.0 , Atl 75 DJF SAM Diff = 0.0
Pac 05 DJF SAM Upper = 0.0 , Pac 05 DJF SAM Lower = 0.0 , Pac 05 DJF SAM Diff = -0.0
Pac 15 DJF SAM Upper = 0.0 , Pac 15 DJF SAM Lower = 0.01 , Pac 15 DJF SAM Diff = -0.0
Pac 25 DJF SAM Upper = 0.01 , Pac 25 DJF SAM Lower = 0.02 , Pac 25 DJF SAM Diff = -0.01
Pac 35 DJF S

## Calculate the Percent Contribution of each Tagged Region's Precip relative to the Global/Total Precip.
## Then Print out the Percentages and percent change at WDC

In [16]:
#taking the time mean of variable first
djf_tp_contribution_Atl_05_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_05_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_15_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_15_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_25_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_25_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_35_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_35_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_45_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_45_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_55_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_55_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_65_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_65_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_75_w_upper_SAM_ctrl = (djf_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Atl_75_w_lower_SAM_ctrl = (djf_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_05_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_05_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_15_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_15_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_25_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_25_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_35_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_35_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_45_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_45_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_55_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_55_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_65_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_65_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_75_w_upper_SAM_ctrl = (mam_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Atl_75_w_lower_SAM_ctrl = (mam_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_05_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_05_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_15_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_15_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_25_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_25_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_35_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_35_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_45_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_45_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_55_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_55_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_65_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_65_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_75_w_upper_SAM_ctrl = (jja_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Atl_75_w_lower_SAM_ctrl = (jja_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_05_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_05_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_15_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_15_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_25_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_25_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_35_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_35_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_45_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_45_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_55_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_55_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_65_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_65_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_75_w_upper_SAM_ctrl = (son_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Atl_75_w_lower_SAM_ctrl = (son_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_05_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_05_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_05_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_05_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_15_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_15_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_15_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_15_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_25_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_25_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_25_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_25_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_35_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_35_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_35_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_35_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_45_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_45_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_45_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_45_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_55_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_55_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_55_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_55_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_65_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_65_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_65_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_65_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_75_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_75_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Atl_75_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Atl_75_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100

#taking the time mean of variable first
djf_tp_contribution_Pac_05_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_05_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_15_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_15_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_25_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_25_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_35_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_35_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_45_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_45_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_55_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_55_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_65_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_65_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_75_w_upper_SAM_ctrl = (djf_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Pac_75_w_lower_SAM_ctrl = (djf_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_05_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_05_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_15_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_15_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_25_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_25_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_35_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_35_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_45_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_45_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_55_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_55_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_65_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_65_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_75_w_upper_SAM_ctrl = (mam_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Pac_75_w_lower_SAM_ctrl = (mam_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_05_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_05_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_15_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_15_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_25_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_25_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_35_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_35_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_45_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_45_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_55_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_55_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_65_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_65_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_75_w_upper_SAM_ctrl = (jja_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Pac_75_w_lower_SAM_ctrl = (jja_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_05_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_05_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_15_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_15_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_25_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_25_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_35_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_35_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_45_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_45_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_55_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_55_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_65_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_65_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_75_w_upper_SAM_ctrl = (son_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Pac_75_w_lower_SAM_ctrl = (son_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_05_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_05_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_05_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_05_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_15_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_15_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_15_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_15_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_25_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_25_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_25_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_25_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_35_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_35_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_35_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_35_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_45_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_45_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_45_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_45_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_55_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_55_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_55_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_55_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_65_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_65_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_65_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_65_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_75_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_75_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Pac_75_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Pac_75_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100

#taking the time mean of variable first
djf_tp_contribution_Ind_05_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_05_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_15_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_15_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_25_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_25_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_35_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_35_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_45_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_45_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_55_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_55_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_65_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_65_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_75_w_upper_SAM_ctrl = (djf_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_Ind_75_w_lower_SAM_ctrl = (djf_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_05_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_05_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_15_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_15_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_25_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_25_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_35_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_35_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_45_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_45_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_55_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_55_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_65_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_65_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_75_w_upper_SAM_ctrl = (mam_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_Ind_75_w_lower_SAM_ctrl = (mam_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_05_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_05_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_15_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_15_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_25_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_25_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_35_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_35_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_45_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_45_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_55_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_55_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_65_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_65_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_75_w_upper_SAM_ctrl = (jja_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_Ind_75_w_lower_SAM_ctrl = (jja_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_05_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_05_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_15_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_15_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_25_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_25_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_35_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_35_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_45_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_45_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_55_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_55_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_65_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_65_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_75_w_upper_SAM_ctrl = (son_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_Ind_75_w_lower_SAM_ctrl = (son_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_05_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_05_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_05_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_05_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_15_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_15_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_15_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_15_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_25_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_25_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_25_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_25_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_35_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_35_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_35_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_35_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_45_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_45_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_45_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_45_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_55_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_55_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_55_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_55_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_65_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_65_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_65_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_65_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_75_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_75_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_Ind_75_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_Ind_75_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100



djf_tp_contribution_AA_land_w_upper_SAM_ctrl = (djf_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_AA_land_w_lower_SAM_ctrl = (djf_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_AA_land_w_upper_SAM_ctrl = (mam_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_AA_land_w_lower_SAM_ctrl = (mam_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_AA_land_w_upper_SAM_ctrl = (jja_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_AA_land_w_lower_SAM_ctrl = (jja_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_AA_land_w_upper_SAM_ctrl = (son_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_AA_land_w_lower_SAM_ctrl = (son_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_AA_land_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_AA_land_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_AA_land_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_AA_land_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100

djf_tp_contribution_AA_ice_w_upper_SAM_ctrl = (djf_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
djf_tp_contribution_AA_ice_w_lower_SAM_ctrl = (djf_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0)/djf_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_AA_ice_w_upper_SAM_ctrl = (mam_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
mam_tp_contribution_AA_ice_w_lower_SAM_ctrl = (mam_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0)/mam_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_AA_ice_w_upper_SAM_ctrl = (jja_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
jja_tp_contribution_AA_ice_w_lower_SAM_ctrl = (jja_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0)/jja_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_AA_ice_w_upper_SAM_ctrl = (son_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
son_tp_contribution_AA_ice_w_lower_SAM_ctrl = (son_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0)/son_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_AA_ice_w_upper_SAM_ctrl = (ann_mean_tp_at_WDC_AA_ice_w_upper_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_upper_SAM_ctrl.mean(axis=0))*100
ann_mean_tp_contribution_AA_ice_w_lower_SAM_ctrl = (ann_mean_tp_at_WDC_AA_ice_w_lower_SAM_ctrl.mean(axis=0)/ann_mean_tp_raw_Total_w_lower_SAM_ctrl.mean(axis=0))*100

In [17]:
print('Atl 05 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 15 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 25 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 35 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 45 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 55 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 65 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 75 DJF SAM Upper =',np.round(djf_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 DJF SAM Lower =',np.round(djf_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 DJF SAM Diff =',np.round(djf_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 05 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 15 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 25 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 35 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 45 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 55 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 65 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 75 MAM SAM Upper =',np.round(mam_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 MAM SAM Lower =',np.round(mam_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 MAM SAM Diff =',np.round(mam_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 05 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 15 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 25 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 35 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 45 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 55 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 65 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 75 JJA SAM Upper =',np.round(jja_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 JJA SAM Lower =',np.round(jja_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 JJA SAM Diff =',np.round(jja_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 05 SON SAM Upper =',np.round(son_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 SON SAM Lower =',np.round(son_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 SON SAM Diff =',np.round(son_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 15 SON SAM Upper =',np.round(son_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 SON SAM Lower =',np.round(son_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 SON SAM Diff =',np.round(son_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 25 SON SAM Upper =',np.round(son_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 SON SAM Lower =',np.round(son_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 SON SAM Diff =',np.round(son_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 35 SON SAM Upper =',np.round(son_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 SON SAM Lower =',np.round(son_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 SON SAM Diff =',np.round(son_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 45 SON SAM Upper =',np.round(son_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 SON SAM Lower =',np.round(son_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 SON SAM Diff =',np.round(son_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 55 SON SAM Upper =',np.round(son_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 SON SAM Lower =',np.round(son_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 SON SAM Diff =',np.round(son_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 65 SON SAM Upper =',np.round(son_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 SON SAM Lower =',np.round(son_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 SON SAM Diff =',np.round(son_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 75 SON SAM Upper =',np.round(son_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 SON SAM Lower =',np.round(son_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 SON SAM Diff =',np.round(son_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 05 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 05 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 15 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 15 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 25 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 25 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 35 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 35 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 45 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 45 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 55 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 55 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 65 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 65 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Atl 75 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Atl 75 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Atl_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Atl_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))

print('')
print('Pac 05 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 15 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 25 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 35 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 45 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 55 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 65 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 75 DJF SAM Upper =',np.round(djf_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 DJF SAM Lower =',np.round(djf_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 DJF SAM Diff =',np.round(djf_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 05 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 15 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 25 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 35 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 45 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 55 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 65 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 75 MAM SAM Upper =',np.round(mam_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 MAM SAM Lower =',np.round(mam_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 MAM SAM Diff =',np.round(mam_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 05 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 15 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 25 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 35 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 45 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 55 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 65 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 75 JJA SAM Upper =',np.round(jja_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 JJA SAM Lower =',np.round(jja_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 JJA SAM Diff =',np.round(jja_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 05 SON SAM Upper =',np.round(son_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 SON SAM Lower =',np.round(son_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 SON SAM Diff =',np.round(son_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 15 SON SAM Upper =',np.round(son_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 SON SAM Lower =',np.round(son_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 SON SAM Diff =',np.round(son_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 25 SON SAM Upper =',np.round(son_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 SON SAM Lower =',np.round(son_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 SON SAM Diff =',np.round(son_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 35 SON SAM Upper =',np.round(son_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 SON SAM Lower =',np.round(son_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 SON SAM Diff =',np.round(son_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 45 SON SAM Upper =',np.round(son_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 SON SAM Lower =',np.round(son_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 SON SAM Diff =',np.round(son_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 55 SON SAM Upper =',np.round(son_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 SON SAM Lower =',np.round(son_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 SON SAM Diff =',np.round(son_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 65 SON SAM Upper =',np.round(son_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 SON SAM Lower =',np.round(son_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 SON SAM Diff =',np.round(son_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 75 SON SAM Upper =',np.round(son_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 SON SAM Lower =',np.round(son_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 SON SAM Diff =',np.round(son_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 05 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 05 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 15 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 15 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 25 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 25 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 35 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 35 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 45 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 45 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 55 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 55 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 65 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 65 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Pac 75 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Pac 75 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Pac_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Pac_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))

print('')
print('Ind 05 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 15 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 25 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 35 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 45 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 55 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 65 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 75 DJF SAM Upper =',np.round(djf_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 DJF SAM Lower =',np.round(djf_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 DJF SAM Diff =',np.round(djf_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 05 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 15 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 25 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 35 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 45 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 55 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 65 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 75 MAM SAM Upper =',np.round(mam_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 MAM SAM Lower =',np.round(mam_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 MAM SAM Diff =',np.round(mam_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 05 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 15 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 25 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 35 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 45 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 55 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 65 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 75 JJA SAM Upper =',np.round(jja_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 JJA SAM Lower =',np.round(jja_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 JJA SAM Diff =',np.round(jja_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 05 SON SAM Upper =',np.round(son_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 SON SAM Lower =',np.round(son_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 SON SAM Diff =',np.round(son_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 15 SON SAM Upper =',np.round(son_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 SON SAM Lower =',np.round(son_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 SON SAM Diff =',np.round(son_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 25 SON SAM Upper =',np.round(son_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 SON SAM Lower =',np.round(son_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 SON SAM Diff =',np.round(son_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 35 SON SAM Upper =',np.round(son_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 SON SAM Lower =',np.round(son_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 SON SAM Diff =',np.round(son_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 45 SON SAM Upper =',np.round(son_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 SON SAM Lower =',np.round(son_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 SON SAM Diff =',np.round(son_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 55 SON SAM Upper =',np.round(son_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 SON SAM Lower =',np.round(son_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 SON SAM Diff =',np.round(son_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 65 SON SAM Upper =',np.round(son_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 SON SAM Lower =',np.round(son_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 SON SAM Diff =',np.round(son_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 75 SON SAM Upper =',np.round(son_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 SON SAM Lower =',np.round(son_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 SON SAM Diff =',np.round(son_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 05 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 05 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_05_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_05_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 15 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 15 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_15_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_15_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 25 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 25 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_25_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_25_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 35 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 35 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_35_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_35_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 45 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 45 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_45_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_45_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 55 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 55 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_55_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_55_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 65 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 65 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_65_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_65_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('Ind 75 ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', Ind 75 ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_Ind_75_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_Ind_75_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))

print('')
print('AA Land DJF SAM Upper =',np.round(djf_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land DJF SAM Lower =',np.round(djf_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land DJF SAM Diff =',np.round(djf_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Land MAM SAM Upper =',np.round(mam_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land MAM SAM Lower =',np.round(mam_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land MAM SAM Diff =',np.round(mam_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Land JJA SAM Upper =',np.round(jja_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land JJA SAM Lower =',np.round(jja_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land JJA SAM Diff =',np.round(jja_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Land SON SAM Upper =',np.round(son_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land SON SAM Lower =',np.round(son_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land SON SAM Diff =',np.round(son_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Land ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Land ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_AA_land_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_AA_land_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))

print('')
print('AA Ice DJF SAM Upper =',np.round(djf_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice DJF SAM Lower =',np.round(djf_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice DJF SAM Diff =',np.round(djf_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - djf_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Ice MAM SAM Upper =',np.round(mam_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice MAM SAM Lower =',np.round(mam_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice MAM SAM Diff =',np.round(mam_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - mam_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Ice JJA SAM Upper =',np.round(jja_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice JJA SAM Lower =',np.round(jja_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice JJA SAM Diff =',np.round(jja_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - jja_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Ice SON SAM Upper =',np.round(son_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice SON SAM Lower =',np.round(son_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice SON SAM Diff =',np.round(son_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - son_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))
print('AA Ice ANN MEAN SAM Upper =',np.round(ann_mean_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice ANN MEAN SAM Lower =',np.round(ann_mean_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2),
      ', AA Ice ANN MEAN SAM Diff =',np.round(ann_mean_tp_contribution_AA_ice_w_upper_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values - ann_mean_tp_contribution_AA_ice_w_lower_SAM_ctrl.sel(lat=-78.63, lon=247.5, method='nearest').values,2))


Atl 05 DJF SAM Upper = 0.07 , Atl 05 DJF SAM Lower = 0.08 , Atl 05 DJF SAM Diff = -0.01
Atl 15 DJF SAM Upper = 0.5 , Atl 15 DJF SAM Lower = 0.52 , Atl 15 DJF SAM Diff = -0.02
Atl 25 DJF SAM Upper = 1.02 , Atl 25 DJF SAM Lower = 0.99 , Atl 25 DJF SAM Diff = 0.03
Atl 35 DJF SAM Upper = 1.74 , Atl 35 DJF SAM Lower = 1.58 , Atl 35 DJF SAM Diff = 0.15
Atl 45 DJF SAM Upper = 2.28 , Atl 45 DJF SAM Lower = 1.92 , Atl 45 DJF SAM Diff = 0.35
Atl 55 DJF SAM Upper = 1.7 , Atl 55 DJF SAM Lower = 2.37 , Atl 55 DJF SAM Diff = -0.67
Atl 65 DJF SAM Upper = 1.24 , Atl 65 DJF SAM Lower = 1.19 , Atl 65 DJF SAM Diff = 0.05
Atl 75 DJF SAM Upper = 0.55 , Atl 75 DJF SAM Lower = 0.38 , Atl 75 DJF SAM Diff = 0.16
Atl 05 MAM SAM Upper = 0.05 , Atl 05 MAM SAM Lower = 0.05 , Atl 05 MAM SAM Diff = -0.0
Atl 15 MAM SAM Upper = 0.21 , Atl 15 MAM SAM Lower = 0.24 , Atl 15 MAM SAM Diff = -0.03
Atl 25 MAM SAM Upper = 0.74 , Atl 25 MAM SAM Lower = 0.87 , Atl 25 MAM SAM Diff = -0.14
Atl 35 MAM SAM Upper = 1.51 , Atl 35 MAM

Ind 35 JJA SAM Upper = 5.56 , Ind 35 JJA SAM Lower = 5.54 , Ind 35 JJA SAM Diff = 0.02
Ind 45 JJA SAM Upper = 6.44 , Ind 45 JJA SAM Lower = 5.34 , Ind 45 JJA SAM Diff = 1.1
Ind 55 JJA SAM Upper = 5.86 , Ind 55 JJA SAM Lower = 3.89 , Ind 55 JJA SAM Diff = 1.96
Ind 65 JJA SAM Upper = 0.56 , Ind 65 JJA SAM Lower = 0.35 , Ind 65 JJA SAM Diff = 0.21
Ind 75 JJA SAM Upper = 0.01 , Ind 75 JJA SAM Lower = 0.01 , Ind 75 JJA SAM Diff = 0.0
Ind 05 SON SAM Upper = 0.12 , Ind 05 SON SAM Lower = 0.13 , Ind 05 SON SAM Diff = -0.02
Ind 15 SON SAM Upper = 0.52 , Ind 15 SON SAM Lower = 0.66 , Ind 15 SON SAM Diff = -0.14
Ind 25 SON SAM Upper = 1.97 , Ind 25 SON SAM Lower = 2.5 , Ind 25 SON SAM Diff = -0.53
Ind 35 SON SAM Upper = 5.59 , Ind 35 SON SAM Lower = 6.37 , Ind 35 SON SAM Diff = -0.78
Ind 45 SON SAM Upper = 8.14 , Ind 45 SON SAM Lower = 8.17 , Ind 45 SON SAM Diff = -0.02
Ind 55 SON SAM Upper = 7.59 , Ind 55 SON SAM Lower = 5.7 , Ind 55 SON SAM Diff = 1.89
Ind 65 SON SAM Upper = 0.7 , Ind 65 SON SA